In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df1.csv')
df2 = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df2.csv')
df3 = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df3.csv')
df4 = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df4.csv')
df1.head()

,Unnamed: 0,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms,city_encoded,state_encoded
0,7,71600.0,3.0,2.0,Ponce,Puerto Rico,1050.0,3484.8,4534.8,5.0,2.470931e+05,639828.768925
1,8,100000.0,2.0,1.0,Ponce,Puerto Rico,1092.0,3920.4,5012.4,3.0,2.706910e+05,624287.881810
2,9,300000.0,5.0,3.0,Las Marias,Puerto Rico,5403.0,324957.6,330360.6,8.0,4.605162e+05,633746.471690
3,10,89000.0,3.0,2.0,Isabela,Puerto Rico,1106.0,583268.4,584374.4,5.0,1.556159e+06,633746.471690
4,11,150000.0,3.0,2.0,Juana Diaz,Puerto Rico,1045.0,3484.8,4529.8,5.0,3.085957e+05,660368.340881


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(self.columns, axis=1)


processing_pipeline = Pipeline(steps=[
    ('Dropper',ColumnDropper(columns=['Unnamed: 0','city','state'])),
    ('Scaler', RobustScaler())
])

In [5]:
df1_N = processing_pipeline.fit_transform(df1)
df1_P = pd.DataFrame(df1_N)
df1_P.columns = df1.columns[1:10]
df1_P.columns

Index(['price', 'bed', 'bath', 'city', 'state', 'house_size', 'sqft_lot',
       'Total_Size', 'Total_Rooms'],
      dtype='object')

In [6]:
df2_N = processing_pipeline.fit_transform(df2)
df2_P = pd.DataFrame(df2_N)
df2_P.columns = df1.columns[1:10]
df2_P.columns

Index(['price', 'bed', 'bath', 'city', 'state', 'house_size', 'sqft_lot',
       'Total_Size', 'Total_Rooms'],
      dtype='object')

In [7]:
df3_N = processing_pipeline.fit_transform(df3)
df3_P = pd.DataFrame(df3_N)
df3_P.columns = df1.columns[1:10]
df3_P.columns

Index(['price', 'bed', 'bath', 'city', 'state', 'house_size', 'sqft_lot',
       'Total_Size', 'Total_Rooms'],
      dtype='object')

In [8]:
df4_N = processing_pipeline.fit_transform(df4)
df4_P = pd.DataFrame(df4_N)
df4_P.columns = df1.columns[1:10]
df4_P.columns

Index(['price', 'bed', 'bath', 'city', 'state', 'house_size', 'sqft_lot',
       'Total_Size', 'Total_Rooms'],
      dtype='object')

In [9]:
df1_P.head()

,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms
0,-0.618049,0.0,0.0,-0.243056,-0.211268,-0.238449,0.0,-0.562200,0.977885
1,-0.548780,-1.0,-1.0,-0.194444,-0.197183,-0.223362,-2.0,-0.474533,0.883823
2,-0.060976,2.0,1.0,4.795139,10.183099,10.054173,3.0,0.230672,0.941072
3,-0.575610,0.0,0.0,-0.178241,18.535211,18.078300,0.0,4.301013,0.941072
4,-0.426829,0.0,0.0,-0.248843,-0.211268,-0.238607,0.0,-0.333716,1.102202


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split



def train_and_evaluate_workflow(all_dfs, train_index):
    """
    Trains a model on one specified DataFrame and tests it on all others.
    """
    
    train_df = all_dfs[train_index]
    test_dfs = [df for i, df in enumerate(all_dfs) if i != train_index]
    
    train_name = f"Dataset {train_index + 1}"
    test_names = [f"Dataset {i + 1}" for i, df in enumerate(all_dfs) if i != train_index]

    print(f"\n--- Starting Workflow: Training on {train_name} ---")
    
    cols_to_drop_for_model = ['price', 'city', 'state', 'Unnamed: 0'] # Adjust if needed
    
    # --- Train the Model ---
    X_train = train_df.drop(columns=cols_to_drop_for_model, errors='ignore')
    y_train = train_df['price']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    print(f"Model training complete for {train_name}.")
    
    # --- Test on the Other Datasets ---
    results = []
    for i in range(len(test_dfs)):
        test_df = test_dfs[i]
        test_name = test_names[i]
        
        X_test = test_df.drop(columns=cols_to_drop_for_model, errors='ignore')
        y_test = test_df['price']
        
        predictions = model.predict(X_test)
        
        # --- THIS IS THE CORRECTED PART ---
        # First, calculate the Mean Squared Error (MSE)
        mse = mean_squared_error(y_test, predictions)
        # Then, take the square root to get the Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mse)
        # --- END OF CORRECTION ---

        print(f"  > Testing on {test_name}... RMSE: ${rmse:,.2f}")
        
        results.append({
            'Model Trained On': train_name,
            'Tested On': test_name,
            'RMSE': rmse
        })
        
    return results
# ==============================================================================
# Main Execution Logic
# ==============================================================================

# First, put your DataFrames into a list
all_datasets = [df1_P, df2_P, df3_P, df4_P]

# Create an empty list to store results from all runs
all_results = []

# Loop four times, once for each dataset as the training set
for i in range(4):
    run_results = train_and_evaluate_workflow(all_datasets, train_index=i)
    all_results.extend(run_results) # Use extend to add list elements

# --- Final Analysis ---
results_df = pd.DataFrame(all_results)
print("\n--- Final Consolidated Results ---")
print(results_df.round(2))


--- Starting Workflow: Training on Dataset 1 ---
Model training complete for Dataset 1.
  > Testing on Dataset 2... RMSE: $262.43
  > Testing on Dataset 3... RMSE: $3.66
  > Testing on Dataset 4... RMSE: $2.25

--- Starting Workflow: Training on Dataset 2 ---
Model training complete for Dataset 2.
  > Testing on Dataset 1... RMSE: $9.62
  > Testing on Dataset 3... RMSE: $3.65
  > Testing on Dataset 4... RMSE: $2.53

--- Starting Workflow: Training on Dataset 3 ---
Model training complete for Dataset 3.
  > Testing on Dataset 1... RMSE: $8.39
  > Testing on Dataset 2... RMSE: $4.95
  > Testing on Dataset 4... RMSE: $2.54

--- Starting Workflow: Training on Dataset 4 ---
Model training complete for Dataset 4.
  > Testing on Dataset 1... RMSE: $8.28
  > Testing on Dataset 2... RMSE: $4.90
  > Testing on Dataset 3... RMSE: $3.35

--- Final Consolidated Results ---
   Model Trained On  Tested On    RMSE
0         Dataset 1  Dataset 2  262.43
1         Dataset 1  Dataset 3    3.66
2        

In [10]:
df_clean = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df_clean.csv')
df_clean.head()

,Unnamed: 0,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms,city_encoded,state_encoded
0,0,105000.0,3.0,2.0,Adjuntas,Puerto Rico,920.0,5227.2,6147.2,5.0,483700.929515,639828.768925
1,1,80000.0,4.0,2.0,Adjuntas,Puerto Rico,1527.0,3484.8,5011.8,6.0,485973.656788,660368.340881
2,2,67000.0,2.0,1.0,Juana Diaz,Puerto Rico,748.0,6534.0,7282.0,3.0,294161.314116,639828.768925
3,3,145000.0,4.0,2.0,Ponce,Puerto Rico,1800.0,4356.0,6156.0,6.0,247093.140064,639828.768925
4,4,65000.0,6.0,2.0,Mayaguez,Puerto Rico,1200.0,2178.0,3378.0,8.0,234061.274994,624287.881810


In [11]:
df_clean.shape

(2224967, 12)

In [12]:
df_N = processing_pipeline.fit_transform(df_clean)
df_P = pd.DataFrame(df_N)
df_P.columns = df_clean.columns[1:10]
df_P.columns

Index(['price', 'bed', 'bath', 'city', 'state', 'house_size', 'sqft_lot',
       'Total_Size', 'Total_Rooms'],
      dtype='object')

In [13]:
df_P

,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms
0,-0.571429,0.0,0.0,-0.556383,-0.196078,-0.228416,0.0,0.293716,0.771332
1,-0.636364,1.0,0.0,0.089362,-0.274510,-0.277683,1.0,0.302306,0.851572
2,-0.670130,-1.0,-1.0,-0.739362,-0.137255,-0.179174,-2.0,-0.422687,0.771332
3,-0.467532,1.0,0.0,0.379787,-0.235294,-0.228034,1.0,-0.600590,0.771332
4,-0.675325,3.0,0.0,-0.258511,-0.333333,-0.348577,3.0,-0.649847,0.710619
...,...,...,...,...,...,...,...,...,...
2224962,0.090649,1.0,0.0,2.294681,0.215686,0.284808,1.0,0.167929,0.842951
2224963,0.064935,0.0,0.0,0.184043,-0.235294,-0.236018,0.0,0.141240,0.847279
2224964,0.298701,3.0,1.0,1.869149,0.549020,0.588778,4.0,0.141240,0.847279
2224965,-0.376883,-1.0,-1.0,-0.542553,-0.254902,-0.284556,-2.0,0.141240,0.847279


In [14]:
from sklearn.model_selection import train_test_split
X = df_P.drop(['price'],axis=1)
y = df_P.price
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [15]:
import lightgbm as lgb

In [17]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import time

# --- Assume you have your four, pre-processed DataFrames ---
# For this example, we'll create four sample DataFrames.
# In your code, you would skip this part and just use your actual DataFrames.


# ==============================================================================
# 1. Configuration: Models and Hyperparameters (Scaled for your hardware)
# ==============================================================================
models_to_test = {
    'LightGBM': {
        'model': lgb.LGBMRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.1]
        }
    },
    'XGBoost': {  # <-- Add this block
        'model': xgb.XGBRegressor(random_state=42, verbosity=0, n_jobs=1),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.1],
            'max_depth': [3, 5]
        }
    },
    'Ridge': {
        'model': Ridge(random_state=42),
        'params': {
            'alpha': [1.0, 10.0, 50.0]
        }
    },
    'KNR':{
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors':[5, 10, 20]}  # Fix typo: n_neighbours -> n_neighbors
    }
}

# Define columns to drop to create the final feature set
cols_to_drop_for_model = ['price', 'city', 'state', 'Unnamed: 0']

# Put your DataFrames into a list for easy iteration
all_datasets = [df1_P, df2_P, df3_P, df4_P]
all_results = []
start_time = time.time()

# ==============================================================================
# 2. Main Execution Loops
# ==============================================================================
# Outer Loop: Designates the training dataset
for i in range(len(all_datasets)):
    train_df = all_datasets[i]
    test_dfs = [df for j, df in enumerate(all_datasets) if j != i]
    
    train_name = f"Dataset {i + 1}"
    print(f"\n{'='*70}")
    print(f"=====   STARTING WORKFLOW: TRAINING ON {train_name.upper()}   =====")
    print(f"{'='*70}")

    X_train = train_df.drop(columns=cols_to_drop_for_model, errors='ignore')
    y_train = train_df['price']

    # Inner Loop: Iterates through each model type
    for model_name, config in models_to_test.items():
        print(f"\n--- Tuning and Training Model: {model_name} on {train_name} ---")
        
        # 3. Tune and Train using GridSearchCV
        # n_jobs=1 is CRITICAL for your hardware to prevent memory overload
        grid_search = GridSearchCV(estimator=config['model'], 
                                   param_grid=config['params'], 
                                   cv=3, # 3-fold CV is sufficient for tuning
                                   scoring='neg_mean_squared_error',
                                   n_jobs=1, # This forces it to run on one core. Slower, but safer.
                                   verbose=1)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        best_params_str = str(grid_search.best_params_)
        print(f"Best params for {model_name} on {train_name}: {best_params_str}")
        
        # 4. Cross-Evaluate the best-tuned model
        for j in range(len(test_dfs)):
            test_df_index = [k for k in range(len(all_datasets)) if k != i][j]
            test_df = all_datasets[test_df_index]
            test_name = f"Dataset {test_df_index + 1}"

            X_test = test_df.drop(columns=cols_to_drop_for_model, errors='ignore')
            y_test = test_df['price']

            predictions = best_model.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)

            print(f"  > Testing on {test_name}... RMSE: ${rmse:,.2f}")
            
            # 5. Data Collection
            all_results.append({
                'Model Trained On': train_name,
                'Model Type': model_name,
                'Tested On': test_name,
                'Best Params': best_params_str,
                'RMSE': rmse
            })

# ==============================================================================
# 6. Saving the Results to a File
# ==============================================================================
results_df = pd.DataFrame(all_results)
results_filename = 'tuned_model_evaluation_results.csv'
results_df.to_csv(results_filename, index=False)

total_time = time.time() - start_time
print(f"\n\nExperiment finished in {total_time / 60:.2f} minutes.")
print(f"All results have been saved to '{results_filename}'")

# ==============================================================================
# Final Analysis
# ==============================================================================
model_performance = results_df.groupby('Model Type')['RMSE'].mean().sort_values()
dataset_performance = results_df.groupby('Model Trained On')['RMSE'].mean().sort_values()

print(f"\n\n{'='*60}\n===============   FINAL ANALYSIS   ===============\n{'='*60}")
print("\n--- Average Performance by Model Type (Lower RMSE is Better) ---")
print(model_performance.round(2))
print("\n--- Average Generalization by Training Dataset (Lower RMSE is Better) ---")
print(dataset_performance.round(2))


=====   STARTING WORKFLOW: TRAINING ON DATASET 1   =====

--- Tuning and Training Model: LightGBM on Dataset 1 ---
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 690
[LightGBM] [Info] Number of data points in the train set: 370826, number of used features: 6
[LightGBM] [Info] Start training from score 0.422221
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 732
[LightGBM] [Info] Number of data points in the train set: 370827, number of used features: 6
[LightGBM] [Info] Start training from score 0.833224
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testi

In [16]:
df_final = pd.merge(left=df3_P,right=df4_P,how='outer')
df_final.head()

,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms
0,-0.92105,-1.0,-1.0,-0.997868,0.3125,0.129959,-2.0,0.974830,-0.758321
1,-0.92105,-1.0,-1.0,-0.742004,0.0000,-0.127357,-2.0,-0.842343,-0.759636
2,-0.92105,-1.0,-1.0,-0.720682,48.8750,45.103298,-2.0,-0.144770,-0.608207
3,-0.92105,-1.0,-1.0,-0.703625,-0.2500,-0.353922,-2.0,-0.266008,-0.759636
4,-0.92105,-1.0,-1.0,-0.669510,-1.0000,-1.043709,-2.0,-0.217194,-0.387058


In [17]:
df_final.shape

(1112480, 9)

In [24]:
X = df_final.drop(['price'],axis=1)
y = df_final.price
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X.head()

,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms
0,-1.0,-1.0,-0.997868,0.3125,0.129959,-2.0,0.974830,-0.758321
1,-1.0,-1.0,-0.742004,0.0000,-0.127357,-2.0,-0.842343,-0.759636
2,-1.0,-1.0,-0.720682,48.8750,45.103298,-2.0,-0.144770,-0.608207
3,-1.0,-1.0,-0.703625,-0.2500,-0.353922,-2.0,-0.266008,-0.759636
4,-1.0,-1.0,-0.669510,-1.0000,-1.043709,-2.0,-0.217194,-0.387058


In [25]:
from sklearn.linear_model import Ridge
r = Ridge(random_state=42,alpha=1.0)
r.fit(X_train,y_train)

,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,42


In [26]:
r.score(X_test,y_test)

0.1450225479991859

In [28]:
from sklearn.model_selection import GridSearchCV
ridge_param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0, 25.0, 50.0, 100.0]}

# n_jobs=1 is still the safest for your hardware
final_grid_search = GridSearchCV(Ridge(random_state=42), 
                                 ridge_param_grid, 
                                 cv=5, # Using 5 folds is standard for final tuning
                                 scoring='neg_mean_squared_error',
                                 n_jobs=1)

final_grid_search.fit(X_train, y_train)

best_alpha = final_grid_search.best_params_['alpha']
print(f"Final tuning complete. The best alpha for the combined dataset is: {best_alpha}")


Final tuning complete. The best alpha for the combined dataset is: 0.1


In [29]:
final_model = Ridge(alpha=best_alpha, random_state=42)

# Train the model on the ENTIRE combined dataset
final_model.fit(X_train, y_train)

print("Final model has been trained successfully!")


Final model has been trained successfully!


In [31]:
final_model.score(X_test,y_test)

0.14502231737180937

In [10]:
import pandas as pd
import numpy as np
import time

# Model Imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
import xgboost as xgb

# Helper Imports
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error # Used internally by GridSearchCV

# ==============================================================================
# 1. Configuration: 5 Models and Their "Hardware-Friendly" Grids
# ==============================================================================
models_to_test = {
    # 'Ridge': {
    #     'model': Ridge(random_state=42),
    #     'params': {
    #         'alpha': [1.0, 10.0, 100.0]
    #     }
    # },
    # 'ElasticNet': {
    #     'model': ElasticNet(random_state=42),
    #     'params': {
    #         'alpha': [1.0, 10.0],
    #         'l1_ratio': [0.5, 0.9]
    #     }
    # },
    # 'DecisionTree': {
    #     'model': DecisionTreeRegressor(random_state=42),
    #     'params': {
    #         'max_depth': [10, 20],
    #         'min_samples_leaf': [10, 20]
    #     }
    # },
    'RandomForest':{
        'model': RandomForestRegressor(),
        'params': {'n_estimators':[50,100,150,200],
                }
    },
    # 'LightGBM': {
    #     'model': lgb.LGBMRegressor(random_state=42),
    #     'params': {
    #         'n_estimators': [100, 200],
    #         'learning_rate': [0.1]
    #     }
    # },
    # 'XGBoost': {
    #     'model': xgb.XGBRegressor(random_state=42, n_jobs=1),
    #     'params': {
    #         'n_estimators': [100],
    #         'learning_rate': [0.1]
    #     }
    # }
}

# Define columns to drop to create the final feature set
cols_to_drop_for_model = ['price', 'city', 'state', 'Unnamed: 0']

# Put your DataFrames into a list for easy iteration
# Ensure df1_P, df2_P, df3_P, and df4_P exist as variables before running
all_datasets = [df1_P, df2_P, df3_P, df4_P]
all_results = []
start_time = time.time()

# ==============================================================================
# 2. Main Execution Loops
# ==============================================================================
# Outer Loop: Designates the training dataset
for i in range(len(all_datasets)):
    train_df = all_datasets[i]
    test_dfs = [df for j, df in enumerate(all_datasets) if j != i]
    
    train_name = f"Dataset {i + 1}"
    print(f"\n{'='*70}\n=====   STARTING WORKFLOW: TRAINING ON {train_name.upper()}   =====\n{'='*70}")

    X_train = train_df.drop(columns=cols_to_drop_for_model, errors='ignore')
    y_train = train_df['price']

    # Inner Loop: Iterates through each model type
    for model_name, config in models_to_test.items():
        print(f"\n--- Tuning and Training Model: {model_name} on {train_name} ---")
        
        # Tune and Train using GridSearchCV
        # n_jobs=1 is critical for your hardware to prevent memory overload
        grid_search = GridSearchCV(estimator=config['model'], 
                                   param_grid=config['params'], 
                                   cv=3,
                                   scoring='neg_mean_squared_error',
                                   n_jobs=1, # This forces it to run on one core. Slower, but safer.
                                   verbose=1)
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        best_params_str = str(grid_search.best_params_)
        print(f"Best params for {model_name} on {train_name}: {best_params_str}")
        
        # Cross-Evaluate the best-tuned model
        for j in range(len(test_dfs)):
            test_df_index = [k for k in range(len(all_datasets)) if k != i][j]
            test_df = all_datasets[test_df_index]
            test_name = f"Dataset {test_df_index + 1}"

            X_test = test_df.drop(columns=cols_to_drop_for_model, errors='ignore')
            y_test = test_df['price']

            # KEY CHANGE: Use model.score() to get R²
            r2_score = best_model.score(X_test, y_test)

            print(f"  > Testing on {test_name}... R² Score: {r2_score:.4f}")
            
            # Data Collection
            all_results.append({
                'Model Trained On': train_name,
                'Model Type': model_name,
                'Tested On': test_name,
                'Best Params': best_params_str,
                'R2_Score': r2_score
            })

# ==============================================================================
# 3. Saving the Results to a File
# ==============================================================================
results_df = pd.DataFrame(all_results)
results_filename = 'r2_score_model_evaluation.csv'
results_df.to_csv(results_filename, index=False)

total_time = time.time() - start_time
print(f"\n\nExperiment finished in {total_time / 60:.2f} minutes.")
print(f"All results have been saved to '{results_filename}'")

# ==============================================================================
# Final Analysis (Modified for R² Score)
# ==============================================================================
# Analysis 1: Find the Best Model Type
model_performance = results_df.groupby('Model Type')['R2_Score'].mean().sort_values(ascending=False) # Higher is better

# Analysis 2: Find the Best Training Dataset
dataset_performance = results_df.groupby('Model Trained On')['R2_Score'].mean().sort_values(ascending=False) # Higher is better


print(f"\n\n{'='*60}\n===============   FINAL ANALYSIS   ===============\n{'='*60}")
print("\n--- Average Performance by Model Type (Higher R² is Better) ---")
print(model_performance.round(4))
print("\n--- Average Generalization by Training Dataset (Higher R² is Better) ---")
print(dataset_performance.round(4))


=====   STARTING WORKFLOW: TRAINING ON DATASET 1   =====

--- Tuning and Training Model: RandomForest on Dataset 1 ---
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best params for RandomForest on Dataset 1: {'n_estimators': 50}
  > Testing on Dataset 2... R² Score: -4893.7055
  > Testing on Dataset 3... R² Score: 0.0631
  > Testing on Dataset 4... R² Score: -0.2293

=====   STARTING WORKFLOW: TRAINING ON DATASET 2   =====

--- Tuning and Training Model: RandomForest on Dataset 2 ---
Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: 

In [11]:
rfr = RandomForestRegressor(n_estimators=50)
rfr.fit(X_train,y_train)

,n_estimators,50
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [12]:
rfr.score(X_test,y_test)

-0.16888051291221085

In [10]:
import pandas as pd
import numpy as np
import time

# --- Deep Learning and Helper Imports ---
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor

# ==============================================================================
# 1. Define the Neural Network Architecture
# ==============================================================================
def create_nn_model(input_shape):
    """Creates a standard sequential Keras model for regression."""
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# ==============================================================================
# 2. Main Evaluation Workflow
# ==============================================================================
# IMPORTANT: This script assumes df1_P, df2_P, df3_P, and df4_P
# already exist as variables in your notebook or script environment.

# Put your scaled DataFrames into a list for easy iteration
all_datasets = [df1_P, df2_P, df3_P, df4_P]
all_results = []
start_time = time.time()

# Loop through each DataFrame to use it as a training set
for i in range(len(all_datasets)):
    train_df = all_datasets[i]
    test_dfs = [df for j, df in enumerate(all_datasets) if j != i]
    
    train_name = f"Dataset {i + 1}"
    print(f"\n{'='*70}\n=====   TRAINING NEURAL NETWORK ON {train_name.upper()}   =====\n{'='*70}")

    # Prepare the training data
    X_train = train_df.drop(columns=['price'])
    y_train = train_df['price']
    
    # Define the KerasRegressor wrapper
    nn_regressor = KerasRegressor(
        model=create_nn_model,
        model__input_shape=(X_train.shape[1],), 
        # --- MODIFICATION FOR SPEED ---
        epochs=20,       # Reduced from 50 to 20 for a much faster run
        batch_size=64,   # Increased from 32 to 64 for fewer updates per epoch
        # --- END OF MODIFICATION ---
        verbose=0,
    )
    
    # Train the neural network on the current training set
    print(f"Fitting model on {train_name}...")
    nn_regressor.fit(X_train, y_train)
    print("Training complete.")
    
    # Cross-Evaluate the trained model
    for j in range(len(test_dfs)):
        test_df_index = [k for k in range(len(all_datasets)) if k != i][j]
        test_df = all_datasets[test_df_index]
        test_name = f"Dataset {test_df_index + 1}"

        X_test = test_df.drop(columns=['price'])
        y_test = test_df['price']

        # Use the .score() method to get the R² value
        r2_score = nn_regressor.score(X_test, y_test)
        print(f"  > Testing on {test_name}... R² Score: {r2_score:.4f}")
        
        # Collect the results
        all_results.append({
            'Model Trained On': train_name,
            'Model Type': 'NeuralNetwork',
            'Tested On': test_name,
            'R2_Score': r2_score
        })

# ==============================================================================
# 3. Saving and Final Analysis
# ==============================================================================
results_df = pd.DataFrame(all_results)
results_filename = 'nn_direct_evaluation_results_faster.csv'
results_df.to_csv(results_filename, index=False)

total_time = time.time() - start_time
print(f"\n\nExperiment finished in {total_time / 60:.2f} minutes.")
print(f"All results have been saved to '{results_filename}'")

# Final analysis to find the most generalizable dataset
dataset_performance = results_df.groupby('Model Trained On')['R2_Score'].mean().sort_values(ascending=False)

print(f"\n\n{'='*60}\n===============   FINAL ANALYSIS   ===============\n{'='*60}")
print("\n--- Average Generalization by Training Dataset (Higher R² is Better) ---")
print(dataset_performance.round(4))


=====   TRAINING NEURAL NETWORK ON DATASET 1   =====
Fitting model on Dataset 1...


e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete.
  > Testing on Dataset 2... R² Score: 0.0575
  > Testing on Dataset 3... R² Score: 0.3047
  > Testing on Dataset 4... R² Score: 0.3736

=====   TRAINING NEURAL NETWORK ON DATASET 2   =====
Fitting model on Dataset 2...


e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete.
  > Testing on Dataset 1... R² Score: 0.0627
  > Testing on Dataset 3... R² Score: -18.0200
  > Testing on Dataset 4... R² Score: 0.2423

=====   TRAINING NEURAL NETWORK ON DATASET 3   =====
Fitting model on Dataset 3...


e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete.
  > Testing on Dataset 1... R² Score: 0.0616
  > Testing on Dataset 2... R² Score: 0.0449
  > Testing on Dataset 4... R² Score: 0.4964

=====   TRAINING NEURAL NETWORK ON DATASET 4   =====
Fitting model on Dataset 4...


e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete.
  > Testing on Dataset 1... R² Score: 0.0689
  > Testing on Dataset 2... R² Score: 0.0442
  > Testing on Dataset 3... R² Score: 0.3063


Experiment finished in 22.18 minutes.
All results have been saved to 'nn_direct_evaluation_results_faster.csv'


===============   FINAL ANALYSIS   ===============

--- Average Generalization by Training Dataset (Higher R² is Better) ---
Model Trained On
Dataset 1    0.2452
Dataset 3    0.2009
Dataset 4    0.1398
Dataset 2   -5.9050
Name: R2_Score, dtype: float64


In [11]:
import pandas as pd
import numpy as np
import time
import joblib # Used to save the final model

# --- Deep Learning and Helper Imports ---
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split

# ==============================================================================
# 1. Combine Your Most Stable Datasets
# ==============================================================================
print("Combining the most stable datasets (Dataset 3 and Dataset 4)...")
# Based on our analysis, df3_P and df4_P are the most robust.
final_df = pd.concat([df3_P, df4_P], ignore_index=True)

print(f"Final combined training dataset shape: {final_df.shape}")


# ==============================================================================
# 2. Define the Robust Neural Network Architecture
# ==============================================================================
def create_robust_nn_model(input_shape):
    """Creates a more robust Keras model with regularization."""
    model = Sequential()
    # Add L2 regularization to penalize large weights
    model.add(Dense(64, input_shape=input_shape, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3)) # Use dropout to prevent overfitting
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))
    # Final output layer
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


# ==============================================================================
# 3. Final Train-Test Split
# ==============================================================================
print("\nPerforming final train-test split on the combined dataset...")

# Prepare the final X (features) and y (target)
X = final_df.drop(columns=['price'])
y = final_df['price']

# Create the one-time split for final evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


# ==============================================================================
# 4. Train the Final Production Model with Early Stopping
# ==============================================================================
print("\n--- Training the final production model ---")

# Define the KerasRegressor wrapper for our robust model
final_nn_model = KerasRegressor(
    model=create_robust_nn_model,
    model__input_shape=(X_train.shape[1],), 
    epochs=150, # Train for a high number of epochs...
    batch_size=64,
    verbose=0,
    # ...but let EarlyStopping find the best stopping point automatically
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    validation_split=0.2 # Use 20% of training data to monitor performance
)

# Fit the model on the final training data
final_nn_model.fit(X_train, y_train)
print("Final model training complete.")


# ==============================================================================
# 5. Evaluate the Final Model on Unseen Data
# ==============================================================================
print("\n--- Evaluating final model on the untouched test set ---")

# Use the .score() method to get the final R² value
final_r2_score = final_nn_model.score(X_test, y_test)

print(f"\nFINAL MODEL R² SCORE: {final_r2_score:.4f}")


# ==============================================================================
# 6. Save the Final Model for Future Use
# ==============================================================================
model_filename = 'final_neural_network_model.joblib'
joblib.dump(final_nn_model, model_filename)

print(f"\nFinal trained model has been saved to '{model_filename}'")

Combining the most stable datasets (Dataset 3 and Dataset 4)...
Final combined training dataset shape: (1112480, 9)

Performing final train-test split on the combined dataset...
Training set shape: (889984, 8)
Test set shape: (222496, 8)

--- Training the final production model ---


e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Final model training complete.

--- Evaluating final model on the untouched test set ---

FINAL MODEL R² SCORE: 0.3513

Final trained model has been saved to 'final_neural_network_model.joblib'
